In [1]:
import cPickle as pickle
import csv
from datetime import date, timedelta

In [2]:
day_notes = {}

for r in range(0, 182):
    print r
    filename = "E:/For Song/Data_sets/text_Days/Day" + str(r) + ".obj"
    with open(filename, "r") as loadfile:
        day_notes[r] = pickle.load(loadfile)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181


In [3]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_curve, auc
from sklearn import preprocessing
from sklearn.decomposition import PCA as sklearnPCA

class DayWiseFeatures:
    def __init__(self, day1, day2):
        self.day1 = day1
        self.day2 = day2
        self.data = self.obtain_data()

    def obtain_data(self):    
        if self.day1 < self.day2:
            FINAL_DATA = day_notes[self.day1]
            initial_data = []
            second_day = 0
            for r in range(self.day1+1, self.day2+1):
                print "R: ", r 
                if FINAL_DATA==None:
                    return FINAL_DATA
                initial_data = FINAL_DATA
                second_day = r
                FINAL_DATA = self.merge_data(initial_data, second_day)
                if r ==181:
                    print r
                    print "AUTO!!!"
                    f = DayClassifier("auto", FINAL_DATA)
                    dicatr = {"TFIDF_sans_PCA":f.XTR[1], "TFIDF_with_PCA":f.XTR[0], 
                              "Y":f.train_data[2], "FEATURE_NAMES":f.TDIDF_feature_names,
                              "INDEX":f.train_data[0]}
                    dicate = {"TFIDF_sans_PCA":f.XTE[1], "TFIDF_with_PCA":f.XTE[0], "Y":f.test_data[2],
                              "FEATURE_NAMES":f.TDIDF_feature_names, "INDEX":f.test_data[0]}
                    print "PROPERTY!!!"
                    f = DayClassifier("property", FINAL_DATA)
                    dicptr = {"TFIDF_sans_PCA":f.XTR[1], "TFIDF_with_PCA":f.XTR[0], 
                              "Y":f.train_data[2], "FEATURE_NAMES":f.TDIDF_feature_names, "INDEX":f.train_data[0]}
                    dicpte = {"TFIDF_sans_PCA":f.XTE[1], "TFIDF_with_PCA":f.XTE[0], "Y":f.test_data[2],
                              "FEATURE_NAMES":f.TDIDF_feature_names, "INDEX":f.test_data[0]}
            return [dicatr, dicate, dicptr, dicpte]
        else:
            print "ERR: Day 1 should be greater than Day 2"
            return None
            
    def merge_data(self, data1, day2):
        DATA1 = data1
        DATA2 = day_notes[day2]
        if (DATA1 != None) and (DATA2 != None):
            FINAL_DATA = [[], [], []]
            DATA2_dict = dict(zip(DATA2[0], DATA2[1]))
            for r in range(0, len(DATA1[0])):
                try:
                    result = DATA2_dict[DATA1[0][r]]
                    FINAL_DATA[0].append(DATA1[0][r])
                    FINAL_DATA[1].append(DATA1[1][r] + " " + result)
                    FINAL_DATA[2].append(DATA1[2][r])
                except KeyError:
                    FINAL_DATA[0].append(DATA1[0][r])
                    FINAL_DATA[1].append(DATA1[1][r])
                    FINAL_DATA[2].append(DATA1[2][r])
            return FINAL_DATA
        else:
            return None

In [ ]:
class DayClassifier:
    def __init__(self, set_name, data):
        self.set_type = set_name
        self.data = data
        if self.data == None:
            print "Wrong paramters!!!"
            return
        print "File loaded"
        self.train_data = self.extract_data("train")
        print "Training data extracted"
        self.test_data = self.extract_data("test")
        print "Test data extracted"
        self.TDIDF_feature_names = []
        self.tfidf_calculator_unigram = TfidfVectorizer(decode_error='ignore', stop_words='english', 
                                                        ngram_range=(1,1), max_features=4000)
        self.scalers = preprocessing.StandardScaler()
        self.sklearn_pca = sklearnPCA(n_components=500)  
        self.XTR = self.train_features()
        self.XTE = self.test_features()
        
    def extract_claim_numbers(self, filename):
        loadfile = open(filename, "r")
        x = loadfile.readlines()
        loadfile.close()
        claim_list = []
        for c in x:
            claim_list.append(c.split("\n")[0])
        return claim_list

    def extract_touchpoint(self, filename):
        loadfile = open(filename, "r")
        x = loadfile.readlines()
        loadfile.close()
        touch = {}
        for c in x:
            touch[c.split(",")[0]] = int((c.split(",")[1]).split("\n")[0])
        return touch

    def extract_data(self, operation_data_type):
        reference_set = []
        touch_set = []
        if self.set_type=="auto":
            if operation_data_type=="train":
                reference_set = self.extract_claim_numbers("E:/For Song/Data_Sets/auto_tr")
                touch_set = self.extract_touchpoint("E:/For Song/Data_Sets/sat_is_1_or_2/auto_tr")
            else:
                reference_set = self.extract_claim_numbers("E:/For Song/Data_Sets/auto_te")
                touch_set = self.extract_touchpoint("E:/For Song/Data_Sets/sat_is_1_or_2/auto_te")
        elif self.set_type=="property":
            if operation_data_type=="train":
                reference_set = self.extract_claim_numbers("E:/For Song/Data_Sets/prop_tr")
                touch_set = self.extract_touchpoint("E:/For Song/Data_Sets/sat_is_1_or_2/prop_tr")
            else:
                reference_set = self.extract_claim_numbers("E:/For Song/Data_Sets/prop_te")
                touch_set = self.extract_touchpoint("E:/For Song/Data_Sets/sat_is_1_or_2/prop_te")
        else:
            print "No such option!!!!"
            return []
        merged = self.data[1]
        claims = self.data[0]
        claims_and_features = dict(zip(claims, merged))
        
        extracted_data = [[], [], []]      
        
        for r in range(0, len(reference_set)):
            try:
                result = claims_and_features[reference_set[r]]
                extracted_data[0].append(reference_set[r])
                extracted_data[1].append(result)
                extracted_data[2].append(touch_set[reference_set[r]])

            except KeyError:
                extracted_data[0].append(reference_set[r])
                extracted_data[1].append("Problem-in-note-processing")
                extracted_data[2].append(touch_set[reference_set[r]])

        return extracted_data

    def train_features(self):
        print "TF-IDF Vectorization started"
        unigram_features = self.tfidf_calculator_unigram.fit_transform(self.train_data[1])
        self.TDIDF_feature_names = self.tfidf_calculator_unigram.get_feature_names()
        print "TF-IDF done"                                       
        XTR = self.scalers.fit_transform(unigram_features.toarray())
        Xtr = self.sklearn_pca.fit_transform(XTR)
        return [Xtr, unigram_features, XTR]

    def test_features(self):
        print "TF-IDF Vectorization started"
        unigram_features = self.tfidf_calculator_unigram.transform(self.test_data[1])
        print "TF-IDF done"
        XTE = self.scalers.transform(unigram_features.toarray())    
        Xte = self.sklearn_pca.transform(XTE)
        return [Xte, unigram_features, XTE]                                                    

In [ ]:
f = DayWiseFeatures(0, 181)

R:  1
R:  2
R:  3
R:  4
R:  5
R:  6
R:  7
R:  8
R:  9
R:  10
R:  11
R:  12
R:  13
R:  14
R:  15
R:  16
R:  17
R:  18
R:  19
R:  20
R:  21
R:  22
R:  23
R:  24
R:  25
R:  26
R:  27
R:  28
R:  29
R:  30
R:  31
R:  32
R: 

In [ ]:
f.data[0]["FEATURE_NAMES"]

In [ ]:
import csv
header = ["CLAIM_NUMBER"] + f.data[0]["FEATURE_NAMES"] + ["TOUCHPOINT"]
claim_num = np.asarray([f.data[0]["INDEX"]])
touchpoint = np.asarray([f.data[0]["Y"]])
to_be_saved = np.concatenate((claim_num.T, f.data[0]["TFIDF_sans_PCA"].todense()), axis=1)
to_be_saved = np.concatenate((to_be_saved, touchpoint.T), axis=1)
oa = to_be_saved.tolist()
with open("E:/For Song/Text_Features_Original_Auto_tr.csv", "w") as savefile:
    writer = csv.writer(savefile,lineterminator="\n")
    writer.writerow(header)
    for r in oa:
        writer.writerow(r)

In [ ]:
import csv
header = ["CLAIM_NUMBER"] + f.data[1]["FEATURE_NAMES"] + ["TOUCHPOINT"]
claim_num = np.asarray([f.data[1]["INDEX"]])
touchpoint = np.asarray([f.data[1]["Y"]])
to_be_saved = np.concatenate((claim_num.T, f.data[1]["TFIDF_sans_PCA"].todense()), axis=1)
to_be_saved = np.concatenate((to_be_saved, touchpoint.T), axis=1)
oa = to_be_saved.tolist()
with open("E:/For Song/Text_Features_Original_Auto_te.csv", "w") as savefile:
    writer = csv.writer(savefile, lineterminator="\n")
    writer.writerow(header)
    for r in oa:
        writer.writerow(r)

In [ ]:
import csv
header = ["CLAIM_NUMBER"] + f.data[2]["FEATURE_NAMES"] + ["TOUCHPOINT"]
claim_num = np.asarray([f.data[2]["INDEX"]])
touchpoint = np.asarray([f.data[2]["Y"]])
to_be_saved = np.concatenate((claim_num.T, f.data[2]["TFIDF_sans_PCA"].todense()), axis=1)
to_be_saved = np.concatenate((to_be_saved, touchpoint.T), axis=1)
oa = to_be_saved.tolist()
with open("E:/For Song/Text_Features_Original_Prop_tr.csv", "w") as savefile:
    writer = csv.writer(savefile, lineterminator="\n")
    writer.writerow(header)
    for r in oa:
        writer.writerow(r)

In [ ]:
import csv
header = ["CLAIM_NUMBER"] + f.data[3]["FEATURE_NAMES"] + ["TOUCHPOINT"]
claim_num = np.asarray([f.data[3]["INDEX"]])
touchpoint = np.asarray([f.data[3]["Y"]])
to_be_saved = np.concatenate((claim_num.T, f.data[3]["TFIDF_sans_PCA"].todense()), axis=1)
to_be_saved = np.concatenate((to_be_saved, touchpoint.T), axis=1)
oa = to_be_saved.tolist()
with open("E:/For Song/Text_Features_Original_Prop_te.csv", "w") as savefile:
    writer = csv.writer(savefile, lineterminator="\n")
    writer.writerow(header)
    for r in oa:
        writer.writerow(r)

In [ ]:
import csv
claim_num = np.asarray([f.data[0]["INDEX"]])
touchpoint = np.asarray([f.data[0]["Y"]])
to_be_saved = np.concatenate((claim_num.T, f.data[0]["TFIDF_with_PCA"]), axis=1)
to_be_saved = np.concatenate((to_be_saved, touchpoint.T), axis=1)
oa = to_be_saved.tolist()
with open("E:/For Song/Text_Features_PCA_Auto_tr.csv", "w") as savefile:
    writer = csv.writer(savefile, lineterminator="\n")
    for r in oa:
        writer.writerow(r)

In [ ]:
import csv
claim_num = np.asarray([f.data[1]["INDEX"]])
touchpoint = np.asarray([f.data[1]["Y"]])
to_be_saved = np.concatenate((claim_num.T, f.data[1]["TFIDF_with_PCA"]), axis=1)
to_be_saved = np.concatenate((to_be_saved, touchpoint.T), axis=1)
oa = to_be_saved.tolist()
with open("E:/For Song/Text_Features_PCA_Auto_te.csv", "w") as savefile:
    writer = csv.writer(savefile, lineterminator="\n")
    for r in oa:
        writer.writerow(r)

In [ ]:
import csv
claim_num = np.asarray([f.data[2]["INDEX"]])
touchpoint = np.asarray([f.data[2]["Y"]])
to_be_saved = np.concatenate((claim_num.T, f.data[2]["TFIDF_with_PCA"]), axis=1)
to_be_saved = np.concatenate((to_be_saved, touchpoint.T), axis=1)
oa = to_be_saved.tolist()
with open("E:/For Song/Text_Features_PCA_Prop_tr.csv", "w") as savefile:
    writer = csv.writer(savefile, lineterminator="\n")
    for r in oa:
        writer.writerow(r)

In [ ]:
import csv
claim_num = np.asarray([f.data[3]["INDEX"]])
touchpoint = np.asarray([f.data[3]["Y"]])
to_be_saved = np.concatenate((claim_num.T, f.data[3]["TFIDF_with_PCA"]), axis=1)
to_be_saved = np.concatenate((to_be_saved, touchpoint.T), axis=1)
oa = to_be_saved.tolist()
with open("E:/For Song/Text_Features_PCA_Prop_te.csv", "w") as savefile:
    writer = csv.writer(savefile, lineterminator="\n")
    for r in oa:
        writer.writerow(r)